Test each tick by itself, depending on time of day. Best:

In [2]:
#Import functions
import polars as pl
from functools import partial
from itertools import product, combinations
import multiprocessing
from numba import jit
import numpy as np
import pickle

In [3]:
#Load dataframe from CSV
currentDf=pl.read_csv("../Data/SP500/minuteHist2021/tradingHours/AAPL.csv", infer_schema_length=None)

In [6]:
#Split into 1 day segments
dayDfs1 = []
for k in range(currentDf['time'][0], currentDf['time'][-1], 86400000):
    current=k
    tempDf=currentDf.filter((pl.col("time")) >= current)
    tempDf=tempDf.filter((pl.col("time")) < current+86400000)
    dayDfs1.append(tempDf)
len(dayDfs1)

964

In [15]:
counter=[]
counterx=0
for k in range(len(dayDfs1)):
    counter.append(len(dayDfs1[k]))
    if len(dayDfs1[k])==390:
        counterx+=1
print(set(counter))
counterx

{0, 390, 332, 307, 373}


664

In [ ]:
#Algo
#Numba for speed
@jit(nopython=True)
#Takes in list of paramaters and an array with all the data
def algo(paramsList, dataArray):
    fee=paramsList[0][0] #Percent change in price to look for
    timeout=int(paramsList[0][1]) #The timeout when it takes too long
    divisor=int(paramsList[0][2]) #Divisor for ema
    allParams=paramsList[1:]
    openPrices=dataArray[:,0] #Open prices
    
    #params has an array of arrays with ticks before, change, version
    
    tradePercent=0
    totalReturn=1

    total=0

    start=60 #Start of data
    end=int(len(openPrices)-61) #End of data

    for j in range(start, end):

        #Check that its not within the first or last hour of trading
        if 0<dataArray[j][2]-dataArray[j-60][2]<4500000 and 0<dataArray[j+60][2]-dataArray[j][2]<4500000:
        
        #Check that its all within one day of trading
        #if 0<dataArray[j][2]-dataArray[j-(gap*len(params))][2]<(60000*gap*len(params)*1.25) and 0<dataArray[j+timeout][2]-dataArray[j][2]<(60000*timeout*1.25):

        #Check that its all within first or last hour of trading
        #if 4500000<dataArray[j][2]-dataArray[j-60][2] or 4500000<dataArray[j+60][2]-dataArray[j][2]:

            #Set start price and continue
            startPrice=dataArray[j][0]
            continueParams=True

            if continueParams:

                #Add to tally
                total+=1

                #Return on investment = new price / old price

                returnValue = (dataArray[j+timeout][0]-fee)/dataArray[j][0]
                totalReturn*=returnValue
                tradePercent+=returnValue

    if total==0:
        tradePercent=1    
    else:
        tradePercent=tradePercent/total
        
    #Return the average trade percent, the total number of times, and the parameters
    returnList = [tradePercent, totalReturn, total, fee, timeout, divisor] + [param for param in allParams for param in param]
    return returnList
    # print(tradePercent)
    # print(tradePerDay)